In [ ]:
import json
from pathlib import Path
from tempfile import gettempdir

from libratom.cli.subcommands import emldump

### Input json object

In [ ]:
eml_export_input = [
    {
        "filename": "andrea_ring_000_1_1.pst",
        "sha256": "37580a9f1c36b4848147181e62b0f6da27d7e914c6c0369a81269c5e44230002",
        "id_list": ["2128516", "2128548", "2128644"],
    },
]

In [ ]:
# Write it to a file to pass to emldump()
json_file_path = Path(gettempdir()) / "eml_export_input.json"

with json_file_path.open(mode="w") as json_file:
    json.dump(eml_export_input, json_file)

### Location of input PST file(s)

In [ ]:
location = Path('data/RevisedEDRMv1_Complete/andrea_ring/')

### Output directory

In [ ]:
out = Path.cwd()

### Ratom subcommand

In [ ]:
status = emldump(out=out, location=location, src=json_file_path)